In [71]:
import pickle
import cv2
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

In [72]:
class DCGAN():

    def __init__(
        self,                   
        dataroot,               # Root directory for dataset
        logfolder,
        workers = 2,            # Number of workers for dataloader
        batch_size = 128,       # Batch size during training
        image_size = 64,        # Spatial size of training images. All images will be resized to this size using a transformer.
        nc = 3,                 # Number of channels in the training images. For color images this is 3
        nz = 100,               # Size of z latent vector (i.e. size of generator input)
        ngf = 64,               # Size of feature maps in generator
        ndf = 64,               # Size of feature maps in discriminator
        num_epochs = 50,        # Number of training epochs
        lr = 0.0002,            # Learning rate for optimizers
        beta1 = 0.5,            # Beta1 hyperparam for Adam optimizers
        ngpu = 1                # Number of GPUs available. Use 0 for CPU mode.
        ):
        # Hyperparameters etc.
        self.logfolder = logfolder
        self.device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
        self.LEARNING_RATE = lr  # could also use two lrs, one for gen and one for disc
        self.BATCH_SIZE = batch_size
        self.IMAGE_SIZE = image_size
        self.CHANNELS_IMG = nc
        self.NOISE_DIM = nz
        self.NUM_EPOCHS = num_epochs
        self.FEATURES_DISC = ndf
        self.FEATURES_GEN = ngf
        self.BETA = beta1

        self.transform = transforms.Compose(
            [
                transforms.Resize((self.IMAGE_SIZE, self.IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(self.CHANNELS_IMG)], [0.5 for _ in range(self.CHANNELS_IMG)]
                ),
            ]
        )

        # comment mnist above and uncomment below if train on CelebA
        self.dataset = datasets.ImageFolder(root=dataroot, transform=self.transform)
        self.dataloader = DataLoader(self.dataset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=workers)
        self.gen = self.Generator(self.NOISE_DIM, self.CHANNELS_IMG, self.FEATURES_GEN).to(self.device)
        self.disc = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC).to(self.device)
        if (self.device.type == 'cuda') and (ngpu > 1):
            self.gen = nn.DataParallel(self.gen, list(range(ngpu)))
            self.disc = nn.DataParallel(self.disc, list(range(ngpu)))
        self.gen.apply(self.weights_init)
        self.disc.apply(self.weights_init)

        # Initialize BCELoss function
        self.criterion = nn.BCELoss()
        # Create batch of latent vectors that we will use to visualize the progression of the generator
        self.fixed_noise = torch.randn(64, self.NOISE_DIM, 1, 1, device=self.device)
        # Establish convention for real and fake labels during training
        self.real_label = 1.
        self.fake_label = 0.
        # Setup Adam optimizers for both G and D
        self.opt_gen = optim.Adam(self.gen.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))
        self.opt_disc = optim.Adam(self.disc.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))


    def weights_init(self, model):
        # Initializes weights according to the DCGAN paper
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif isinstance(m, (nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
                nn.init.constant_(m.bias.data, 0)

    def train(self):

        # Training Loop

        # Lists to keep track of progress
        img_list = []
        G_losses = []
        D_losses = []

        toPrintEpoch = list(range(0, self.NUM_EPOCHS, max((self.NUM_EPOCHS // 100), 1)))

        # For each epoch
        for epoch in tqdm(range(self.NUM_EPOCHS)):
            # For each batch in the dataloader
            for data in self.dataloader:

                ############################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                ###########################
                ## Train with all-real batch
                self.disc.zero_grad()
                # Format batch
                real_cpu = data[0].to(self.device)
                b_size = real_cpu.size(0)
                label = torch.full((b_size,), self.real_label, dtype=torch.float, device=self.device)
                # Forward pass real batch through D
                output = self.disc(real_cpu).view(-1)
                # Calculate loss on all-real batch
                errD_real = self.criterion(output, label)
                # Calculate gradients for D in backward pass
                errD_real.backward()
                D_x = output.mean().item()

                ## Train with all-fake batch
                # Generate batch of latent vectors
                noise = torch.randn(b_size, self.NOISE_DIM, 1, 1, device=self.device)
                # Generate fake image batch with G
                fake = self.gen(noise)
                label.fill_(self.fake_label)
                # Classify all fake batch with D
                output = self.disc(fake.detach()).view(-1)
                # Calculate D's loss on the all-fake batch
                errD_fake = self.criterion(output, label)
                # Calculate the gradients for this batch, accumulated (summed) with previous gradients
                errD_fake.backward()
                D_G_z1 = output.mean().item()
                # Compute error of D as sum over the fake and the real batches
                errD = errD_real + errD_fake
                # Update D
                self.opt_disc.step()

                ############################
                # (2) Update G network: maximize log(D(G(z)))
                ###########################
                self.gen.zero_grad()
                label.fill_(self.real_label)  # fake labels are real for generator cost
                # Since we just updated D, perform another forward pass of all-fake batch through D
                output = self.disc(fake).view(-1)
                # Calculate G's loss based on this output
                errG = self.criterion(output, label)
                # Calculate gradients for G
                errG.backward()
                D_G_z2 = output.mean().item()
                # Update G
                self.opt_gen.step()

                # Save Losses for plotting later
                G_losses.append(errG.item())
                D_losses.append(errD.item())
            
            if (epoch in toPrintEpoch):
                # Check how the generator is doing by saving G's output on fixed_noise
                with torch.no_grad():
                    no_grad_fake = self.gen(self.fixed_noise).detach().cpu()

                img_grid_fake = vutils.make_grid(no_grad_fake, padding=2, normalize=True)
                
                plt.figure(figsize=(15,15))
                plt.axis("off")
                plt.imshow(np.transpose(img_grid_fake,(1,2,0)))
                plt.savefig("{}/{}.png".format(self.logfolder, epoch))
                plt.close()

                img_list.append(img_grid_fake)

        return img_list, G_losses, D_losses 

    class Discriminator(nn.Module):
        def __init__(self, channels_img, features_d):
            super(DCGAN.Discriminator, self).__init__()
            self.disc = nn.Sequential(
                # input: N x channels_img x 64 x 64
                nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
                nn.LeakyReLU(0.2),
                # _block(in_channels, out_channels, kernel_size, stride, padding)
                self._block(features_d, features_d * 2, 4, 2, 1),
                self._block(features_d * 2, features_d * 4, 4, 2, 1),
                self._block(features_d * 4, features_d * 8, 4, 2, 1),
                # After all _block img output is 4x4 (Conv2d below makes into 1x1)
                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
                nn.Sigmoid(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2),
            )

        def forward(self, x):
            return self.disc(x)


    class Generator(nn.Module):
        def __init__(self, channels_noise, channels_img, features_g):
            super(DCGAN.Generator, self).__init__()
            self.net = nn.Sequential(
                # Input: N x channels_noise x 1 x 1
                self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
                self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
                self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
                self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
                nn.ConvTranspose2d(
                    features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
                ),
                # Output: N x channels_img x 64 x 64
                nn.Tanh(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

        def forward(self, x):
            return self.net(x)

In [73]:
class Generator(nn.Module):
    def __init__(self, scale_factor):
        upsample_block_num = int(math.log(scale_factor, 2))

        super(Generator, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=4),
            nn.PReLU()
        )
        self.block2 = ResidualBlock(64)
        self.block3 = ResidualBlock(64)
        self.block4 = ResidualBlock(64)
        self.block5 = ResidualBlock(64)
        self.block6 = ResidualBlock(64)
        self.block7 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )
        block8 = [UpsampleBLock(64, 2) for _ in range(upsample_block_num)]
        block8.append(nn.Conv2d(64, 3, kernel_size=9, padding=4))
        self.block8 = nn.Sequential(*block8)

    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)
        block6 = self.block6(block5)
        block7 = self.block7(block6)
        block8 = self.block8(block1 + block7)

        return (torch.tanh(block8) + 1) / 2

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.bn1(residual)
        residual = self.prelu(residual)
        residual = self.conv2(residual)
        residual = self.bn2(residual)

        return x + residual
    
class UpsampleBLock(nn.Module):
    def __init__(self, in_channels, up_scale):
        super(UpsampleBLock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * up_scale ** 2, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(up_scale)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.prelu(x)
        return x

In [74]:
def preprocess_one_image(image_path, device):
    image = cv2.imread(image_path).astype(np.float32) / 255.0

    # BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert image data to pytorch format data
    tensor = image_to_tensor(image, False, False).unsqueeze_(0)

    # Transfer tensor channel image format data to CUDA device
    tensor = tensor.to(device=device, memory_format=torch.channels_last, non_blocking=True)

    return tensor

def image_to_tensor(image, range_norm, half):
    # Convert image data type to Tensor data type
    tensor = torch.from_numpy(np.ascontiguousarray(image)).permute(2, 0, 1).float()

    # Scale the image data from [0, 1] to [-1, 1]
    if range_norm:
        tensor = tensor.mul(2.0).sub(1.0)

    # Convert torch.float32 image data type to torch.half image data type
    if half:
        tensor = tensor.half()

    return tensor

def tensor_to_image(tensor, range_norm, half):
    if range_norm:
        tensor = tensor.add(1.0).div(2.0)
    if half:
        tensor = tensor.half()

    image = tensor.squeeze(0).permute(1, 2, 0).mul(255).clamp(0, 255).cpu().numpy().astype("uint8")

    return image

In [75]:
collection = 'Collection2'

In [76]:
device = torch.device("cuda" if (torch.cuda.is_available() and 1 > 0) else "cpu")
with open('models/DCGAN/{}/gen'.format(collection), 'rb') as f:
    gen = pickle.load(f)
with torch.no_grad():
    no_grad_fake = gen(torch.randn(64, 100, 1, 1, device=device)).detach().cpu()
img_grid_fake = vutils.make_grid(no_grad_fake[:1], normalize=True)
vutils.save_image(img_grid_fake, 'result/1_NFT_outDCGAN.png')




img = cv2.imread('result/1_NFT_outDCGAN.png', cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.fastNlMeansDenoisingColored(img,img,10,10,7,21)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite('result/2_NFT_outCV2.png', img)





with open('models/SRGAN/{}/gen'.format('lilCollection2'), 'rb') as f:
    gen = pickle.load(f)
lr_tensor = preprocess_one_image('result/2_NFT_outCV2.png', device)
with torch.no_grad():
    sr_tensor = gen(lr_tensor)
img = tensor_to_image(sr_tensor, False, False)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite('result/3_NFT_outSRGAN.png', img)




UPSCALE_FACTOR = 2
model = Generator(UPSCALE_FACTOR).eval()
model.cuda()
model.load_state_dict(torch.load('models/SRGAN/{}/gen.pth'.format('lilCollection2')))
img = Image.open('result/2_NFT_outCV2.png')
with torch.no_grad():
    img = Variable(ToTensor()(img), volatile=True).unsqueeze(0)
img = img.cuda()
img = model(img)
img = ToPILImage()(img[0].data.cpu())
img.save('result/4_NFT_outSRGAN.png')


img = cv2.imread('result/4_NFT_outSRGAN.png', cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.fastNlMeansDenoisingColored(img,img,10,10,7,21)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite('result/5_NFT_outCV2.png', img)

/tmp/ipykernel_44025/1691514626.py:40: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(ToTensor()(img), volatile=True).unsqueeze(0)


True